In [2]:
import numpy as np
import pandas as pd
import pickle

# Warning: depending on file size, running this script requires a lot of RAM. This script was used to merge 34GB's of data, requiring about 60GBs of RAM i

In [3]:
def open_file(i):
    a_file = open("C:/Users/test/Documents/Results/Neighbourhood_results_#"+ str(i) +".pkl", "rb")
    results = pickle.load(a_file)
    a_file.close()
    return results

In [4]:
def open_file_combined(i):
    a_file = open("C:/Users/test/Documents/Results/combined_results_#"+ str(i) +".pkl", "rb")
    results = pickle.load(a_file)
    a_file.close()
    return results

In [5]:
def save_outcome(outcome, i):
    a_file = open("C:/Users/test/Documents/Results/combined_results_#"+ str(i) +".pkl", "wb")
    pickle.dump(outcome, a_file)
    a_file.close()
    print('File number '+ f'{i}\r' + ' saved', end="")

In [6]:
def add_column_names(data):
    for i in data.columns:
        #data[i] = data[i].astype('|S500')
        data[i] = data[i].astype(float, errors = 'ignore')
    return data.rename(columns = {0:'LT-suitability', 1:'MT-suitability',2:'HT-suitability',3:'Best Primary Source', 4:'Cityheating', 5:'Net-temp',6:'Heat pumps',7:'Gas connections',8:'Average spending',9: 'Year', 10:'Municipality', 11: 'Neighbourhood', 12:'HG contract', 13: 'HP contract', 14: 'Insulation contract', 15:'Dominant system' }).sort_values(['Municipality', 'Neighbourhood','Year']).reset_index(drop = True)

In [7]:
def combine_replications(results):
    first = True    
    for i in range(98): 
        total = add_column_names(pd.DataFrame(results[1][i][0]))
        indexers = total.iloc[:,[10,11]]
        for j in range(1,4):
            total = total.append(add_column_names(pd.DataFrame(results[1][i][j]))) 
        means = total.groupby(by = total.index).mean()
        categoricals = total[total.columns[total.dtypes == object]][['Best Primary Source','Net-temp','HG contract', 'HP contract', 'Insulation contract', 'Dominant system']].groupby(by = total.index).agg(lambda x: pd.Series.mode(x)[0])#.agg(lambda x:x.value_counts())
        outcome = indexers.merge(means, left_index = True, right_index = True).merge(categoricals, left_index = True, right_index = True).to_numpy()
        outcome = np.expand_dims(outcome, 0)
        if first == True:
            outcome_set = outcome
            first = False
        else:
            outcome_set = np.append(outcome_set, outcome, axis = 0)
        print('Current rep: '+ f'{i}\r', end="")
    return outcome_set

In [8]:
# number of files
def process_data(i):
    results = open_file(i)
    outcome = combine_replications(results)
    save_outcome(outcome, i)

In [ ]:
# pool = multiprocessing.Pool(12)
# pool.map(process_data, range(0, 15))

In [ ]:
from multiprocessing import Pool

if __name__ == '__main__':
    with Pool(5) as p:
        p.map(process_data, list(range(0,15)))

In [9]:
def combine_data():
    total = open_file_combined(0)
    for i in range(1,15):
        total = np.append(total, open_file_combined(i), axis = 0)
        print('current rep: ' + str(i))
    return total

In [10]:
results = combine_data()

current rep: 1
current rep: 2
current rep: 3
current rep: 4
current rep: 5
current rep: 6
current rep: 7
current rep: 8
current rep: 9
current rep: 10
current rep: 11
current rep: 12
current rep: 13
current rep: 14


In [11]:
scenarios = pd.read_csv('Scenarios.csv')

In [12]:
HG_high_results = results[scenarios['Scenario HG high']]
HP_high_results = results[scenarios['Scenario HP high']]
CO2_high_results = results[scenarios['Scenario CO2 high']]
CO2_low_results = results[scenarios['Scenario CO2 low']]


In [13]:
def make_float(data):
    for i in data.columns:
        data[i] = data[i].astype(float, errors = 'ignore')
    return data


def combine_scenarios(results):
        first = True    

        
        total = make_float(pd.DataFrame(results[0])).sort_values([0,1,9]).reset_index(drop = True)
        indexers = total.iloc[:,[0,1]]
        #for i in range(10):
        for i in range(results.shape[0]):
            total = total.append(make_float(pd.DataFrame(results[i])).sort_values([0,1,9]).reset_index(drop = True))
            print('Current rep: '+ f'{i}\r', end="")
        print('Grouping now')
        means = total.groupby(by = total.index).mean()
        categoricals = total[total.columns[total.dtypes == object]][[10,11,12,13,14,15]].groupby(by = total.index).agg(lambda x: pd.Series.mode(x)[0])#.agg(lambda x:x.value_counts())
        outcome = indexers.merge(means, left_index = True, right_index = True).merge(categoricals, left_index = True, right_index = True).to_numpy()
        
        
#         outcome = np.expand_dims(outcome, 0)
#         if first == True:
#             outcome_set = outcome
#             first = False
#         else:
#             outcome_set = np.append(outcome_set, outcome, axis = 0)
        
        return outcome

In [68]:
from datetime import datetime
def combine_scenarios_2(results):
#         first = True    

        
#         total = make_float(pd.DataFrame(results[0])).sort_values([0,1,9]).reset_index(drop = True)
#         indexers = total.iloc[:,[0,1]]
#         #for i in range(10):
#         for i in range(results.shape[0]):
#             total = total.append(make_float(pd.DataFrame(results[i])).sort_values([0,1,9]).reset_index(drop = True))
#             print('Current rep: '+ f'{i}\r', end="")
        #shape = results.shape
    results = results.reshape(results.shape[0] * results.shape[1], results.shape[2])
    print('Turning into floats for means ', datetime.now())
    results = make_float(pd.DataFrame(results))
    print('Grouping now ', datetime.now())
    means = results.groupby(by = [0, 1, 9]).mean()
    print('Means done ', datetime.now() )
    results[9] = results[9].astype(object)
    categoricals = results[results.columns[results.dtypes == object]].groupby(by = [0,1,9]).agg(lambda x: pd.Series.mode(x)[0])#.agg(lambda x:x.value_counts())
    print('Categoricals done ', datetime.now())
    outcome = means.merge(categoricals, left_index = True, right_index = True)#.to_numpy()
        
        
#         outcome = np.expand_dims(outcome, 0)
#         if first == True:
#             outcome_set = outcome
#             first = False
#         else:
#             outcome_set = np.append(outcome_set, outcome, axis = 0)
        
    return outcome

In [16]:
from datetime import datetime

In [ ]:
start = datetime.now()
HG_scen_high = combine_scenarios(HG_high_results)
print('Combining took ' + str(datetime.now()-start))

In [ ]:
start = datetime.now()
HP_scen_high = combine_scenarios(HP_high_results)
print('Combining took ' + str(datetime.now()-start))

In [17]:
start = datetime.now()
CO2_scen_high = combine_scenarios(CO2_high_results)
print('Combining took ' + str(datetime.now()-start))

Grouping now 794
Combining took 2:10:17.559721


In [18]:
start = datetime.now()
CO2_scen_low = combine_scenarios(CO2_low_results)
print('Combining took ' + str(datetime.now()-start))

Grouping now 153
Combining took 0:05:51.953046


In [19]:
#pd.DataFrame(HP_scen_high).to_csv('HP_high_scenario.csv')
#pd.DataFrame(HG_scen_high).to_csv('HG_high_scenario.csv')
pd.DataFrame(CO2_scen_high).to_csv('CO2_high_scenario.csv')
pd.DataFrame(CO2_scen_low).to_csv('CO2_low_scenario.csv')

In [20]:
HG_scen_high_std = combine_scenarios_std(HG_high_results)
HP_scen_high_std = combine_scenarios_std(HP_high_results)
CO2_scen_high_std = combine_scenarios_std(CO2_high_results)
CO2_scen_low_std = combine_scenarios_std(CO2_low_results)

Grouping now 424
Grouping now 196
Grouping now 794
Grouping now 153


In [21]:
pd.DataFrame(HP_scen_high_std).to_csv('HP_high_scenario_std.csv')
pd.DataFrame(HG_scen_high_std).to_csv('HG_high_scenario_std.csv')
pd.DataFrame(CO2_scen_high_std).to_csv('CO2_high_scenario_std.csv')
pd.DataFrame(CO2_scen_low_std).to_csv('CO2_low_scenario_std.csv')

In [22]:
pd.DataFrame(combine_scenarios_std(results)).to_csv('total_results_std.csv')

Grouping now 1497


In [69]:
means = combine_scenarios_2(results)

Turning into floats for means  2020-07-16 09:54:18.630019
Grouping now  2020-07-16 09:57:10.022557
Means done  2020-07-16 09:57:33.772622
Categoricals done  2020-07-16 10:00:26.381944


In [70]:
means.to_csv('total_results_means.csv')

In [71]:
means

2         3    4         5   \
0               1               9                                           
Amsterdam       AMC             2020.0  0.599838  1.000000  1.0  0.000000   
                                2021.0  0.730164  1.000000  1.0  0.000000   
                                2022.0  0.787939  1.000000  1.0  0.000000   
                                2023.0  0.826257  1.000000  1.0  0.000000   
                                2024.0  0.855587  1.000000  1.0  0.000000   
...                                          ...       ...  ...       ...   
Zwartewaterland Zwartsluis Kern 2056.0  0.554846  0.907405  1.0  0.418482   
                                2057.0  0.555357  0.908402  1.0  0.421973   
                                2058.0  0.555887  0.909399  1.0  0.426805   
                                2059.0  0.556369  0.910283  1.0  0.431887   
                                2060.0  0.556869  0.911172  1.0  0.438656   

                                              6         7             8   \
0               1               9                                          
Amsterdam       AMC             2020.0  0.013324  0.986676  15316.460865   
                                2021.0  0.028334  0.971666  18954.617898   
                                2022.0  0.041741  0.958259  20610.030154   
                                2023.0  0.053343  0.946657  21727.527156   
                                2024.0  0.062987  0.937013  22585.090688   
...                                          ...       ...           ...   
Zwartewaterland Zwartsluis Kern 2056.0  0.077265  0.504253  10510.542078   
                                2057.0  0.078375  0.499651  10562.234911   
                                2058.0  0.079368  0.493827  10616.549584   
                                2059.0  0.080267  0.487847  10670.102803   
                                2060.0  0.081103  0.480241  10724.929879   

                                                          10 11          12  \
0               1               9                                             
Amsterdam       AMC             2020.0  No available options  0  Integrated   
                                2021.0  No available options  0  Integrated   
                                2022.0  No available options  0  Integrated   
                                2023.0  No available options  0  Integrated   
                                2024.0  No available options  0  Integrated   
...                                                      ... ..         ...   
Zwartewaterland Zwartsluis Kern 2056.0  No available options  0  Integrated   
                                2057.0  No available options  0  Integrated   
                                2058.0  No available options  0  Integrated   
                                2059.0  No available options  0  Integrated   
                                2060.0  No available options  0  Integrated   

                                                   13             14  \
0               1               9                                      
Amsterdam       AMC             2020.0  Building Team  Building Team   
                                2021.0  Building Team  Building Team   
                                2022.0  Building Team  Building Team   
                                2023.0  Building Team  Building Team   
                                2024.0  Building Team  Building Team   
...                                               ...            ...   
Zwartewaterland Zwartsluis Kern 2056.0     Integrated     Integrated   
                                2057.0     Integrated     Integrated   
                                2058.0     Integrated     Integrated   
                                2059.0     Integrated     Integrated   
                                2060.0     Integrated     Integrated   

                                                     15  
0               1               